In [ ]:
%pip install langchain 
%pip install langchain_chroma 
%pip install langchain_community 
%pip install langchain_core
%pip install langchain_google_vertexai
%pip install langchain_text_splitters
%pip install jq

In [13]:
# loader = DirectoryLoader('docs2', glob='**/*.htm', loader_cls=BSHTMLLoader, loader_kwargs={'open_encoding': 'utf8'}, show_progress=True, use_multithreading=True)

# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# splits = text_splitter.split_documents(docs)

# model = 'Alibaba-NLP/gte-large-en-v1.5'
# model_kwargs = model_kwargs = {'device':'cpu', 'trust_remote_code': True}
# encode_kwargs = {'normalize_embeddings': True}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [1]:
import langchain_community.document_loaders as document_loaders
kmb_loader = document_loaders.JSONLoader('./bus_cleaned.json', jq_schema=".features", text_content=False)
kmb_loaded = kmb_loader.load()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(kmb_loaded)
splits[0].page_content

"[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.192446, 22.345353]}, 'properties': {'routeId': 1001, 'companyCode': 'KMB', 'district': '', 'routeNameC': '1', 'routeNameS': '1', 'routeNameE': '1', 'routeType': 1, 'serviceMode': 'R', 'specialType': 0, 'journeyTime': 49, 'locStartNameC': '竹園邨', 'locStartNameS': '竹园邨', 'locStartNameE': 'CHUK YUEN ESTATE', 'locEndNameC': '尖沙咀碼頭', 'locEndNameS': '尖沙咀码头', 'locEndNameE': 'STAR FERRY', 'hyperlinkC': 'https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=zh-hk', 'hyperlinkS': 'https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=zh-cn', 'hyperlinkE': 'https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=en', 'fullFare': 6.4, 'lastUpdateDate': '2023-07-07T00:00:00', 'routeSeq': 1, 'stopSeq': 1, 'stopId': 4001, 'stopPickDrop': 2, 'stopNameC': '竹園邨總站', 'stopNameS': '竹园邨总站', 'stopNameE': 'CHUK YUEN ESTATE BUS TERMINUS'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates':"

In [3]:
## Divide into chunks for Chroma add document
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

chunks = list(divide_chunks(splits, 1000))

In [4]:
from langchain_community.embeddings import FakeEmbeddings
fakeEmbeddings = FakeEmbeddings(
    size=764
)

: 

In [5]:
from langchain_chroma import Chroma

vertex_store = Chroma(
    collection_name='KMB',
    embedding_function=fakeEmbeddings,
    persist_directory='./chroma_db_fake',
)

for c, i in enumerate(chunks[0:10]):
    print(vertex_store.add_documents(i))
    print(f"Added {c+1} chunks")

In [ ]:
fakeEmbeddings.embed_query('1001')